## Chronos_Fx-Price Finetuned Multi M


In [ ]:
pip install chronos-forecasting

In [ ]:
import pandas as pd
from chronos import BaseChronosPipeline

pipeline = BaseChronosPipeline.from_pretrained("amazon/chronos-2", device_map="cuda")


In [4]:
# -*- coding: utf-8 -*-
"""
Chronos-2 – Multi-FX walk-forward (monthly, levels) with fine-tuning,
evaluation starts strictly after the fine-tuning period.

Pipeline:
1) Fine-tune Chronos-2 on Norges Bank FX panel (1980–1999):
      NB1980-1999.csv with columns:
      ds; AUD; CAD; CHF; DKK; GBP; ISK; JPY; NZD; SEK; USD; XDR

2) Evaluate on MultiFXData.csv (price-only), but only for months
   strictly after the fine-tune end date:
   - Cut: last business day of previous month
   - Forecast: next calendar month at daily frequency
   - Aggregate to monthly mean over business days
   - Per FX series: Observations, RMSE, MAE, Directional Accuracy,
                    DM test vs Random Walk (MSE, h=1)
   - Output: metrics CSV (one row per FX series)
"""

from __future__ import annotations
import io, time, math
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Callable, List

import numpy as np
import pandas as pd
import requests, certifi
from sklearn.metrics import mean_absolute_error

import torch
from chronos import BaseChronosPipeline  # chronos-forecasting>=2.0


# -----------------------------
# Configuration
# -----------------------------
@dataclass
class Config:
    # Evaluation data (multi-FX)
    url_multi: str = (
        "https://raw.githubusercontent.com/bredeespelid/"
        "Data_MasterOppgave/refs/heads/main/EURNOK/MultiFXData.csv"
    )
    # Fine-tuning panel (Norges Bank 1980–1999)
    url_finetune: str = (
        "https://raw.githubusercontent.com/bredeespelid/"
        "Data_MasterOppgave/refs/heads/main/FineTuneData/NB1980-1999.csv"
    )

    m_freq: str = "M"           # monthly periods, month-end
    min_hist_days: int = 40
    max_context: int = 2048
    max_horizon: int = 64       # must exceed longest month
    retries: int = 3
    timeout: int = 60
    verbose: bool = True
    metrics_csv: str = "FX_Chronos2_finetuned_metrics_monthly_postFT.csv"
    include_fx: Optional[List[str]] = None  # e.g. ["EUR", "USD", "SEK", "DKK", "GBP"]

    # Fine-tuning hyperparameters
    ft_prediction_length: int = 32
    ft_num_steps: int = 50
    ft_learning_rate: float = 1e-5
    ft_batch_size: int = 2
    ft_logging_steps: int = 10


CFG = Config()

# FX columns used for fine-tuning
FINETUNE_FX_COLS = [
    "AUD", "CAD", "CHF", "DKK", "GBP",
    "ISK", "JPY", "NZD", "SEK", "USD", "XDR",
]


# -----------------------------
# Helper: download CSV with retries
# -----------------------------
def download_csv_text(url: str, retries: int, timeout: int) -> str:
    """Download text from URL with retry/backoff logic."""
    last_err = None
    for k in range(1, retries + 1):
        try:
            r = requests.get(url, timeout=timeout, verify=certifi.where())
            r.raise_for_status()
            return r.text
        except Exception as e:
            last_err = e
            if k < retries:
                wait = 1.5 * k
                print(f"[warning] Download failed ({k}/{retries}): {e}. Retrying in {wait:.1f}s...")
                time.sleep(wait)
    raise RuntimeError(f"Download failed: {last_err}")


# -----------------------------
# Load Multi-FX evaluation data
# -----------------------------
def load_multi_fx(url: str) -> pd.DataFrame:
    """
    Load MultiFXData.csv with columns like:
      DATE, I44, AUD, CHF, DKK, EUR, CAD, GBP, ..., USD, ...

    Robust to:
      - comma/semicolon separators
      - dot/comma decimals

    Returns:
        Daily DataFrame indexed by DATE, numeric columns forward-filled.
    """
    text = download_csv_text(url, CFG.retries, CFG.timeout)

    def _try_read(sep: str, decimal: str) -> pd.DataFrame:
        return pd.read_csv(io.StringIO(text), sep=sep, encoding="utf-8-sig", decimal=decimal)

    # First attempt: comma + dot decimals
    raw = _try_read(",", ".")
    if "DATE" not in raw.columns:
        # Second attempt: semicolon + dot decimals
        raw = _try_read(";", ".")
    if "DATE" not in raw.columns:
        # Fallback: comma/semicolon + comma decimals
        for sep in (",", ";"):
            raw = _try_read(sep, ",")
            if "DATE" in raw.columns:
                break

    if "DATE" not in raw.columns:
        raise ValueError(f"Expected a DATE column; got: {list(raw.columns)[:10]} ...")

    raw["DATE"] = pd.to_datetime(raw["DATE"], errors="coerce")
    raw = raw.dropna(subset=["DATE"]).sort_values("DATE").set_index("DATE")

    num_df = raw.apply(pd.to_numeric, errors="coerce")
    daily_idx = pd.date_range(num_df.index.min(), num_df.index.max(), freq="D")
    df_d = num_df.reindex(daily_idx).ffill()
    df_d.index.name = "DATE"
    return df_d


def series_daily_and_b(df_d: pd.DataFrame, col: str) -> Tuple[pd.Series, pd.Series]:
    """
    Extract one FX series from the multi-frame.

    Returns:
        S_b: business-day series (B, forward-filled)
        S_d: calendar-day series (D)
    """
    if col not in df_d.columns:
        raise ValueError(f"Column {col} not found in MultiFXData.")
    S_d = df_d[col].astype(float)
    S_d.name = col
    S_b = S_d.asfreq("B").ffill()
    S_b.name = col
    return S_b, S_d


def last_trading_day(S_b: pd.Series, start: pd.Timestamp, end: pd.Timestamp) -> Optional[pd.Timestamp]:
    """Return the last business day in [start, end]."""
    sl = S_b.loc[start:end]
    return sl.index[-1] if not sl.empty else None


# -----------------------------
# Load fine-tuning FX panel (1980–1999)
# -----------------------------
def load_finetune_fx_panel(url: str) -> pd.DataFrame:
    """
    Load Norges Bank FX panel 1980–1999 for fine-tuning.

    CSV format:
      ds; AUD; CAD; CHF; DKK; GBP; ISK; JPY; NZD; SEK; USD; XDR

    Returns:
        df: index = DATE (daily), columns = FINETUNE_FX_COLS
    """
    text = download_csv_text(url, CFG.retries, CFG.timeout)
    raw = pd.read_csv(
        io.StringIO(text),
        sep=";",
        decimal=".",
        encoding="utf-8-sig",
    )

    required = ["ds"] + FINETUNE_FX_COLS
    missing = set(required) - set(raw.columns)
    if missing:
        raise ValueError(f"Missing columns in fine-tune CSV: {missing}. Got: {list(raw.columns)}")

    df = (
        raw[required]
        .rename(columns={"ds": "DATE"})
        .assign(DATE=lambda x: pd.to_datetime(x["DATE"], dayfirst=True, errors="coerce"))
        .dropna(subset=["DATE"])
        .sort_values("DATE")
        .set_index("DATE")
    )

    for col in FINETUNE_FX_COLS:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(how="all", subset=FINETUNE_FX_COLS)
    return df


# -----------------------------
# Chronos-2: base pipeline + fine-tuning
# -----------------------------
def build_base_chronos_pipeline() -> BaseChronosPipeline:
    """Load the base Chronos-2 pipeline on CUDA (fp16)."""
    if not torch.cuda.is_available():
        raise SystemExit("CUDA not available. Please install a CUDA build of PyTorch and a recent NVIDIA driver.")

    pipeline: BaseChronosPipeline = BaseChronosPipeline.from_pretrained(
        "amazon/chronos-2",
        device_map="cuda",
        torch_dtype=torch.float16,
    )
    return pipeline


def finetune_chronos_on_nb_panel(
    pipeline: BaseChronosPipeline,
    fx_panel: pd.DataFrame,
) -> BaseChronosPipeline:
    """
    Fine-tune Chronos-2 on the Norges Bank FX panel (1980–1999).

    Each FX column (AUD, CAD, ..., XDR) is treated as a separate univariate series.
    No covariates are used for fine-tuning (price-only panel).
    """
    train_inputs: List[Dict] = []

    for col in FINETUNE_FX_COLS:
        series = fx_panel[col].dropna().astype(np.float32).values
        if series.size < CFG.ft_prediction_length * 2:
            continue

        train_inputs.append(
            {
                "target": series,
                "past_covariates": {},
                "future_covariates": {},
            }
        )

    if not train_inputs:
        raise RuntimeError("No valid series found for fine-tuning.")

    if CFG.verbose:
        total_len = sum(len(d["target"]) for d in train_inputs)
        print("\n[Fine-tuning Chronos-2 on Norges Bank FX panel 1980–1999]")
        print(f"  Number of series: {len(train_inputs)}")
        print(f"  Total length across series: {total_len}")
        print(
            f"  prediction_length={CFG.ft_prediction_length}, "
            f"num_steps={CFG.ft_num_steps}, lr={CFG.ft_learning_rate}, "
            f"batch_size={CFG.ft_batch_size}"
        )

    pipeline = pipeline.fit(
        inputs=train_inputs,
        prediction_length=CFG.ft_prediction_length,
        num_steps=CFG.ft_num_steps,
        learning_rate=CFG.ft_learning_rate,
        batch_size=CFG.ft_batch_size,
        logging_steps=CFG.ft_logging_steps,
    )

    return pipeline


# -----------------------------
# Chronos-2 model builder (multi-FX, point forecasts only)
# -----------------------------
def build_model_chronos2_multi(
    pipeline: BaseChronosPipeline,
    max_context: int,
    horizon_len: int,
) -> Callable[[np.ndarray, int], np.ndarray]:
    """
    Build a Chronos-2 forecasting function for generic univariate FX series.

    Returns:
        forecast_fn(x, H) -> np.ndarray length H (daily point forecast)
    """

    def extract_median(pred: pd.DataFrame) -> np.ndarray:
        """Extract median forecast from Chronos output."""
        df = pred.copy()
        if "timestamp" in df.columns:
            df = df.sort_values("timestamp")

        if "0.5" in df.columns:
            arr = df["0.5"].to_numpy()
        elif "predictions" in df.columns:
            arr = df["predictions"].to_numpy()
        elif "forecast" in df.columns and "quantile" in df.columns:
            df = df.loc[df["quantile"] == 0.5].copy()
            arr = df["forecast"].to_numpy()
        else:
            for cand in ("forecast", "p50", "median", "mean"):
                if cand in df.columns:
                    arr = df[cand].to_numpy()
                    break
            else:
                raise RuntimeError(f"Chronos2 predict_df: unsupported schema. Columns={list(df.columns)}")

        return np.asarray(arr, dtype=float)

    def forecast_fn(x: np.ndarray, H: int) -> np.ndarray:
        """
        Forecast H daily steps ahead for a single univariate FX series.
        """
        ctx = np.asarray(x, dtype=float).ravel()[-max_context:]
        ts = pd.date_range("2000-01-01", periods=len(ctx), freq="D")

        df = pd.DataFrame(
            {
                "item_id": "series_1",
                "timestamp": ts,
                "target": ctx,
            }
        )

        with torch.inference_mode():
            pred = pipeline.predict_df(
                df,
                prediction_length=H,
                quantile_levels=[0.5],
                id_column="item_id",
                timestamp_column="timestamp",
                target="target",
            )

        med = extract_median(pred)
        return med[:H]

    return forecast_fn


# -----------------------------
# Monthly walk-forward for one FX series (with start_period)
# -----------------------------
def walk_forward_monthly(
    S_b: pd.Series,
    S_d: pd.Series,
    forecast_fn: Callable[[np.ndarray, int], np.ndarray],
    series_name: str,
    start_period: Optional[pd.Period] = None,
) -> pd.DataFrame:
    """
    Monthly walk-forward:
      - Cut = last B-day in previous month
      - Forecast next month at daily frequency
      - Aggregate to monthly mean over business days

    Only months >= start_period are evaluated if start_period is provided.
    """
    first_m = pd.Period(S_b.index.min(), freq=CFG.m_freq)
    last_m  = pd.Period(S_b.index.max(),  freq=CFG.m_freq)

    if start_period is not None:
        # Ensure we start no earlier than the requested evaluation period.
        first_m = max(first_m, start_period)

    months = pd.period_range(first_m, last_m, freq=CFG.m_freq)

    rows: Dict = {}
    dropped: Dict[str, str] = {}

    for m in months:
        prev_m = m - 1
        m_start, m_end = m.start_time, m.end_time
        prev_start, prev_end = prev_m.start_time, prev_m.end_time

        cut = last_trading_day(S_b, prev_start, prev_end)
        if cut is None:
            dropped[str(m)] = "no_cut_in_prev_m"
            continue

        hist_d = S_d.loc[:cut]
        if hist_d.size < CFG.min_hist_days:
            dropped[str(m)] = f"hist<{CFG.min_hist_days}"
            continue

        idx_m_b = S_b.index[(S_b.index >= m_start) & (S_b.index <= m_end)]
        if idx_m_b.size < 1:
            dropped[str(m)] = "no_bdays_in_m"
            continue
        y_true = float(S_b.loc[idx_m_b].mean())

        H = (m_end.date() - m_start.date()).days + 1
        if H <= 0 or H > CFG.max_horizon:
            dropped[str(m)] = f"horizon_invalid(H={H})"
            continue

        context = min(CFG.max_context, len(hist_d))
        x = hist_d.values[-context:]

        pf = forecast_fn(x, H)
        if pf.shape[0] < H:
            dropped[str(m)] = f"horizon_short({pf.shape[0]})"
            continue

        f_idx = pd.date_range(cut + pd.Timedelta(days=1), periods=H, freq="D")
        pred_daily = pd.Series(pf[:H], index=f_idx, name="point")

        pred_b = pred_daily.reindex(idx_m_b, method=None)
        if pred_b.isna().all():
            dropped[str(m)] = "no_overlap_pred_B_days"
            continue
        y_pred = float(pred_b.dropna().mean())

        rows[str(m)] = {
            "series": series_name,
            "month": m,
            "cut": cut,
            "y_true": y_true,
            "y_pred": y_pred,
        }

    df = pd.DataFrame.from_dict(rows, orient="index")
    if not df.empty:
        df = df.set_index("month").sort_index()

    if CFG.verbose and dropped:
        miss = [str(m) for m in months if m not in df.index]
        if miss:
            print(f"[{series_name}] Dropped months:")
            for mm in miss:
                print(f"  {mm}: {dropped.get(mm, 'unknown')}")

    return df


# -----------------------------
# Evaluation & DM-test
# -----------------------------
def _normal_cdf(z: float) -> float:
    """Standard normal CDF without SciPy."""
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))


def dm_test(y_true: pd.Series, y_model: pd.Series, y_rw: pd.Series, h: int = 1, loss: str = "mse"):
    """Diebold–Mariano test for equal predictive accuracy vs random walk benchmark."""
    df = pd.concat({"y": y_true, "m": y_model, "rw": y_rw}, axis=1).dropna()
    if df.empty or len(df) < 5:
        return float("nan"), float("nan")

    e_m = df["y"] - df["m"]
    e_r = df["y"] - df["rw"]
    d = np.abs(e_m) - np.abs(e_r) if loss.lower() == "mae" else (e_m**2) - (e_r**2)

    N = int(len(d))
    d_mean = float(d.mean())
    gamma0 = float(np.var(d, ddof=1)) if N > 1 else 0.0
    var_bar = gamma0 / N

    if h > 1 and N > 2:
        for k in range(1, min(h - 1, N - 1) + 1):
            w_k = 1.0 - k / h
            cov_k = float(np.cov(d[k:], d[:-k], ddof=1)[0, 1])
            var_bar += 2.0 * w_k * cov_k / N

    if var_bar <= 0 or not np.isfinite(var_bar):
        return float("nan"), float("nan")

    dm_stat = d_mean / math.sqrt(var_bar)
    p_val = 2.0 * (1.0 - _normal_cdf(abs(dm_stat)))
    return dm_stat, p_val


def evaluate_with_dm(eval_df: pd.DataFrame) -> Dict[str, float]:
    """
    Compute RMSE, MAE, directional accuracy and DM-test vs random walk for one FX series.
    """
    df = eval_df.copy()
    df["err"] = df["y_true"] - df["y_pred"]
    core = df.dropna(subset=["y_true", "y_pred"]).copy()

    n_obs = int(len(core))
    rmse = float(np.sqrt(np.mean(np.square(core["err"])))) if n_obs else np.nan
    mae  = float(mean_absolute_error(core["y_true"], core["y_pred"])) if n_obs else np.nan

    core["y_prev"] = core["y_true"].shift(1)
    mask = core["y_prev"].notna()
    dir_true = np.sign(core.loc[mask, "y_true"] - core.loc[mask, "y_prev"])
    dir_pred = np.sign(core.loc[mask, "y_pred"] - core.loc[mask, "y_prev"])
    hits = int((dir_true.values == dir_pred.values).sum())
    total = int(mask.sum())
    dir_acc = (hits / total) if total else np.nan

    df["rw_pred"] = df["y_true"].shift(1)
    dm_stat, p_val = dm_test(df["y_true"], df["y_pred"], df["rw_pred"], h=1, loss="mse")

    return {
        "observations": n_obs,
        "rmse": rmse,
        "mae": mae,
        "dir_hits": hits,
        "dir_total": total,
        "dir_acc": dir_acc,
        "dm_stat": float(dm_stat) if np.isfinite(dm_stat) else np.nan,
        "dm_pvalue": float(p_val) if np.isfinite(p_val) else np.nan,
    }


# -----------------------------
# Main
# -----------------------------
def main():
    # 1) Load evaluation frame (multi-FX)
    df_d = load_multi_fx(CFG.url_multi)

    # Determine which numeric columns to run
    all_cols = [c for c in df_d.columns if pd.api.types.is_numeric_dtype(df_d[c])]
    if CFG.include_fx:
        fx_cols = [c for c in CFG.include_fx if c in all_cols]
    else:
        fx_cols = all_cols

    # 2) Load fine-tuning panel and fine-tune Chronos-2
    fx_panel = load_finetune_fx_panel(CFG.url_finetune)
    ft_start = fx_panel.index.min()
    ft_end = fx_panel.index.max()

    if CFG.verbose:
        print(f"\nFine-tune panel: {ft_start.date()} → {ft_end.date()} | n={len(fx_panel)}")
        print(f"Fine-tune FX columns: {FINETUNE_FX_COLS}")

    base_pipeline = build_base_chronos_pipeline()
    ft_pipeline = finetune_chronos_on_nb_panel(base_pipeline, fx_panel)

    # 3) Define evaluation start period as the first month AFTER fine-tune end month
    #    Example: fine-tune ends 1999-12-31 → eval starts from 2000-01
    eval_start_period = pd.Period(ft_end, freq=CFG.m_freq) + 1
    if CFG.verbose:
        print(f"\nEvaluation starts from period: {eval_start_period} (i.e., strictly after fine-tune period)")

    # 4) Build forecasting function based on fine-tuned pipeline
    forecast_fn = build_model_chronos2_multi(
        pipeline=ft_pipeline,
        max_context=CFG.max_context,
        horizon_len=CFG.max_horizon,
    )

    if CFG.verbose:
        print(f"\nRunning monthly walk-forward for {len(fx_cols)} FX series:", fx_cols)

    # 5) Walk-forward and metrics per FX series (only months >= eval_start_period)
    metrics_rows = []

    for col in fx_cols:
        S_b, S_d = series_daily_and_b(df_d, col)
        if CFG.verbose:
            print(f"\n[{col}] Data (B): {S_b.index.min().date()} → {S_b.index.max().date()} | n={len(S_b)}")

        df_eval = walk_forward_monthly(
            S_b=S_b,
            S_d=S_d,
            forecast_fn=forecast_fn,
            series_name=col,
            start_period=eval_start_period,
        )

        if df_eval.empty or df_eval["y_pred"].isna().all():
            if CFG.verbose:
                print(f"[{col}] No evaluable months after fine-tune period; skipping.")
            continue

        m = evaluate_with_dm(df_eval)
        m["series"] = col
        metrics_rows.append(m)

        if np.isfinite(m["dir_acc"]) and m["dir_total"] > 0:
            print(
                f"[{col}] Obs={m['observations']}, RMSE={m['rmse']:.4f}, MAE={m['mae']:.4f}, "
                f"DirAcc={m['dir_hits']}/{m['dir_total']} ({m['dir_acc']*100:.1f}%), "
                f"DM={m['dm_stat']:.3f}, p={m['dm_pvalue']:.4f}"
            )
        else:
            print(
                f"[{col}] Obs={m['observations']}, RMSE={m['rmse']:.4f}, MAE={m['mae']:.4f}, "
                f"DirAcc=NA, DM={m['dm_stat']:.3f}, p={m['dm_pvalue']:.4f}"
            )

    # 6) Save metrics
    if not metrics_rows:
        print("No series produced metrics after the fine-tune period. Check data and settings.")
        return

    metrics_df = pd.DataFrame(metrics_rows)[
        ["series", "observations", "rmse", "mae", "dir_hits", "dir_total", "dir_acc", "dm_stat", "dm_pvalue"]
    ].sort_values("rmse")
    metrics_df.to_csv(CFG.metrics_csv, index=False, encoding="utf-8-sig")
    print(f"\nSaved metrics to: {CFG.metrics_csv}")


if __name__ == "__main__":
    main()



Fine-tune panel: 1980-12-10 → 1999-12-31 | n=4930
Fine-tune FX columns: ['AUD', 'CAD', 'CHF', 'DKK', 'GBP', 'ISK', 'JPY', 'NZD', 'SEK', 'USD', 'XDR']

[Fine-tuning Chronos-2 on Norges Bank FX panel 1980–1999]
  Number of series: 11
  Total length across series: 52815
  prediction_length=32, num_steps=50, lr=1e-05, batch_size=2


/tmp/ipython-input-2409010327.py:263: FutureWarning: Fine-tuning support is experimental and may be changed in future versions.
  pipeline = pipeline.fit(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,2.024800
20,2.704700
30,1.760400
40,1.748100
50,1.498000



Evaluation starts from period: 2000-01 (i.e., strictly after fine-tune period)

Running monthly walk-forward for 19 FX series: ['I44', 'AUD', 'EUR', 'CAD', 'GBP', 'HKD', 'JPY', 'MYR', 'NZD', 'SGD', 'SEK', 'PLN', 'USD', 'PHP', 'IDR', 'KRW', 'THB', 'TWD', 'XDR']

[I44] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[I44] Obs=300, RMSE=1.4042, MAE=0.6603, DirAcc=190/299 (63.5%), DM=1.532, p=0.1255

[AUD] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[AUD] Obs=300, RMSE=0.0979, MAE=0.0430, DirAcc=194/299 (64.9%), DM=1.259, p=0.2079

[EUR] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[EUR] Obs=300, RMSE=0.1436, MAE=0.0636, DirAcc=198/299 (66.2%), DM=2.034, p=0.0420

[CAD] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[CAD] Obs=300, RMSE=0.1044, MAE=0.0485, DirAcc=192/299 (64.2%), DM=1.231, p=0.2183

[GBP] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[GBP] Obs=300, RMSE=0.2824, MAE=0.1150, DirAcc=170/299 (56.9%), DM=1.177, p=0.2393

[HKD] Data (B): 1999-01-04 → 2024-12-12 | n=6769
[HKD] Obs=300, RMSE=0.0